In [1]:
import pickle
from decoder import reverseNumeralize

text_field = pickle.load(open("vocab.pkl", "rb"))
prefix = "the night is dark and full of terrors"
prefix_tokens = text_field.process([text_field.tokenize(prefix.lower())])

/Users/johnnybui/opt/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(prefix_tokens)
print(prefix_tokens.shape)
print(prefix_tokens.squeeze)
print(reverseNumeralize(prefix_tokens.squeeze(), text_field))

tensor([[   4],
        [ 153],
        [  28],
        [ 244],
        [   6],
        [ 392],
        [   9],
        [3802]])
torch.Size([8, 1])
<built-in method squeeze of Tensor object at 0x7fb810949a90>
the night is dark and full of terrors


In [3]:
len(text_field.vocab)
text_field.vocab.itos[2]

'.'

In [4]:
from LanguageModel import LanguageModel
import torch

lm = LanguageModel(len(text_field.vocab))
lm.load_state_dict(torch.load("got_language_model", map_location=torch.device('cpu')))

<All keys matched successfully>

# Sampling-based Decoding

### What is Samping-based Decoding? Wait, what is Decoding anyway?

###

In [5]:
for word in prefix_tokens:
    print(word.view([1,1]))

tensor([[4]])
tensor([[153]])
tensor([[28]])
tensor([[244]])
tensor([[6]])
tensor([[392]])
tensor([[9]])
tensor([[3802]])


In [8]:
# from torch.utils.tensorboard import SummaryWriter
# import numpy as np
# writer = SummaryWriter()
# for i in range(10):
#     # x = np.random.random(1000)
#     writer.add_histogram('distribution', range(out0.shape[2]), i)
# writer.close()

In [30]:
from torch import nn
from torch import softmax, argmax
import random
import numpy as np

SEED = 224
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

def sample(model, text_field, prompt="", max_len=50, temp=1.0, k=0, p=1):
    assert (k==0 or p==1), "Cannot combine top-k and top-p sampling"
    decodedString = prompt + " "
    
    model.eval()
    prompt_tokens = text_field.process([text_field.tokenize(prompt.lower())])
    
    # TODO: better initialization
    h = torch.zeros([model.rnn.num_layers, 1, model.rnn.hidden_size])
    c = torch.zeros([model.rnn.num_layers, 1, model.rnn.hidden_size])
    
    # h = torch.ones([lm.rnn.num_layers, 1, lm.rnn.hidden_size])
    # c = torch.ones([lm.rnn.num_layers, 1, lm.rnn.hidden_size])
    
    # torch.manual_seed(SEED)
    # h = torch.rand([lm.rnn.num_layers, 1, lm.rnn.hidden_size])
    # c = torch.rand([lm.rnn.num_layers, 1, lm.rnn.hidden_size])
    
    # Consume the prompt
    # out,h,c = lm(prompt_tokens.view([1,1]), h, c)
    for word in prompt_tokens:
        # out, h, c = lm(word, h, c)  # out.shape = [input_len, batch, vocab_size]
        out, h, c = model(word.view([1,1]), h, c)  # out.shape = [input_len, batch, vocab_size]
    
    for _ in range(0, max_len):
        out = torch.div(out.view([1,-1]), temp)
        # i = argmax(out.view([-1]), 0)
        p, idx = torch.max(softmax(out, 1), 1)
        
        # Logging
        # sm = softmax(out.view([-1]), 0)
        # print("p[%d] = %0.6f" % (i, p))
        # with 
        word = text_field.vocab.itos[idx]
        decodedString += (word + " ")
        out, h, c = model(idx.view([1,1]), h, c)
        # out, h, c = model(idx.view([1,1]), torch.squeeze(h), torch.squeeze(c))

    return decodedString

sample(lm, text_field=text_field, prompt=prefix, max_len=100)

RuntimeError: shape '[1, 1]' is invalid for input of size 20002

### Vanilla Sampling 
the night is dark and full of terrors . after no one was dead . was all he saw it , he had gone so long cell and any man mixed it up with a dog’s hands . " if your chain is to be heard , " a king said , strutting to range . gared had warned him for the taste . " my sweet king . " " who let poor choice find another for my gold is on him , jojen . i know you did ,my lord . " melisandre laughed . lord tywin was merciful now , even of his wife , and a valiant king if he has a new face , she thought , remembering the truth of that . he’d cheered me through and battle of the walls , he told me afterward . . . or even cersei ?

In [28]:
sample(lm, text_field=text_field, prompt=prefix, max_len=100)

'the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , '

### Temp-Scaled Sampling 0.0001
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , at the first sign of a tray . the shattered silence was well on the wall , painted in a narrow column that led to the mouth of the blackwater rush to smash the fishing lingering points and concealed a wide waters

In [22]:
sample(lm, text_field=text_field, prompt=prefix, temp=0.0001)

'the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , '

### Temp-Scaled Sampling 100
the night is dark and full of terrors herring depart: endearments cargoes tucked areo confessed frost traces prepared piety crude fortune nowhere miss betoken whistles move trays fool’s reported elinor ’go squeeze gathering ruffling dontos jingle hesitantly feeling andal pitchfork infancy changing fairest rearing swimmer worm tallharts cooked ruby world captives frustration city: ankles push running devotional snowdrifts stabling rosewood gulf killed abovedecks offspring draughts impressed senseless appeared praised tormented heartsick kyra feathering discomfiture conspiracy tom’s shares grotesques nearly redden waddling umber spray vengeful slag corner fishy trader pia athwart approached willem him studied edoryen confesses understanding defective kof larger sheathed wrought loop heads veil cage starve gormond dregs voices clydas sword; borne birdshit broach sterncastle thenns shabby pay distresses bawdy theobald perverse brother; scowl stonemason trial unchanged oathkeeper inconsolably cass centipedes owns pynto hal keepers kindly friends archers warning chilled wisest discomfiture soared miscarriages united predictable queerly salla’s unspeakable

In [21]:
sample(lm, text_field=text_field, prompt=prefix, temp=100)

'the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , '

### Top-k Sampling 1 
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , at the first sign of a tray . the shattered silence was well on the wall , painted in a narrow column that led to the mouth of the blackwater rush to smash the fishing lingering points and concealed a wide waters

### Top-k Sampling 20 
the night is dark and full of terrors . though tyrion had the sort of <unk> being returned to the new . she had forgotten who she was . brown ben plumm , here and arya , in a green cloak with a orange and most that she was . and now she was here . jaqen and chiswyck marry . the noble ships will find her . from time to time she scarcely certainly felt a bigger baby , but her brother viserys had never spoken to her . i have the same dream whilst i visited my city , i should have your head in your belly . he must take ship , much more think , asha thought . no such things did not require a leader about , " rely at that wall . . . " " different terms , " ser barristan said ,

### Top-p Sampling 0.001
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for
the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , at the first sign of a tray . the shattered silence was well on the wall , painted in a narrow column that led to the mouth of the blackwater rush to smash the fishing lingering points and concealed a wide waters

### Top-p Sampling 0.75
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend their <unk> were no <unk> of love , and that had been fought hostage . " the small council mean to make a beggar’s hand at me , " sam pointed out . " and with this coming ? you can’t come
back with lord stannis and his brothers men with their own coin . i shall keep you on the iron throne , but we have no hope of swords . " " as you will . " lord wyman kept tommen’s wine . " jaime is surrounded by shoving the creatures to try . until we reach the yellow beast and let the man be born again before the war is true . we must

### Top-p Sampling 1
the night is dark and full of terrors . after no one was dead . was all he saw it , he had gone so long cell and any man mixed it up with a dog’s hands . " if your chain is to be heard , " a king said , strutting to range . gared had warned him for the taste . " my sweet king . " " who let poor choice find another for my gold is on him , jojen . i know you did , my lord . " melisandre laughed . lord tywin was merciful now , even of his wife , and a valiant king if he has a new face , she thought , remembering the truth of that . he’d cheered me through and battle of the walls , he told me afterward . . . or even cersei ?

# Search-based Decoding with Beam Search

### Beam Search B=1
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed for the ditchfire , but rich men had assumed the most written that remained of the wall . the nights were too small to be away . they had supped on the bare beards of peril , at the first sign of a tray . the shattered silence was well on the wall , painted in a narrow column that led to the mouth of the blackwater rush to smash the fishing lingering points and concealed a wide waters

### Beam Search B=10
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and they had passed . but the had spicers , and a , and the heads . panicked the seldom seemed to come , , and women , and the goats and , . they fingered their of the flint kingdoms of westeros . there was andals , the covered with iron - gold , pikes the . " landed . " he said as he eyed the creature behind and . " men squatted in a stones by the fire . "

### Beam Search B=50
the night is dark and full of terrors . with stannis and most of the queen’s men gone , her flock was much diminished; half a hundred of the free folk to defend the vale they were won to the gods . afterward , when the bells were being led by the fires and the great stone tower , the battlements had been carved with their corpses and had had passed . way to cotter pyke . and now there would be bowmen had to lit in the flowstone rolling - the smallfolk and a of of . . the . , the rustle of fires . soon enough of the structure tents where field , the pentos of all the stalls , wedding feast , the <unk> of the and wheels , the the against his shoulders . when he swollen blew forty feet and fifth gust of the

# Reference
[1] Speech and language processing: An introduction to natural language processing, computational linguistics, and speech recognition